## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,Magadi,12.9667,77.2333,82.45,71,0,9.01,IN,clear sky
1,1,Tromso,69.6496,18.9570,26.62,68,75,18.41,NO,light snow
2,2,Cherskiy,68.7500,161.3000,31.86,100,100,4.27,RU,light snow
3,3,Butaritari,3.0707,172.7902,81.45,80,70,15.35,KI,broken clouds
4,4,Bluff,-46.6000,168.3333,55.80,67,96,6.53,NZ,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,Magadi,12.9667,77.2333,82.45,71,0,9.01,IN,clear sky
3,3,Butaritari,3.0707,172.7902,81.45,80,70,15.35,KI,broken clouds
5,5,Panama City,8.9936,-79.5197,76.95,90,75,3.44,PA,broken clouds
7,7,Thinadhoo,0.5333,72.9333,83.82,73,99,6.15,MV,overcast clouds
11,11,Arraial Do Cabo,-22.9661,-42.0278,75.87,87,0,21.70,BR,clear sky
21,21,Rikitea,-23.1203,-134.9692,77.23,66,0,22.84,PF,clear sky
25,25,Vila Velha,-20.3297,-40.2925,75.94,90,4,5.93,BR,clear sky
27,27,Yulara,-25.2406,130.9889,85.59,25,100,9.44,AU,overcast clouds
28,28,Vaini,-21.2000,-175.2000,78.30,57,53,20.07,TO,broken clouds
30,30,Simao,22.7886,100.9748,83.34,24,2,3.58,CN,clear sky


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Country                205
Weather Description    205
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities = preferred_cities_df.dropna()
clean_preferred_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,Magadi,12.9667,77.2333,82.45,71,0,9.01,IN,clear sky
3,3,Butaritari,3.0707,172.7902,81.45,80,70,15.35,KI,broken clouds
5,5,Panama City,8.9936,-79.5197,76.95,90,75,3.44,PA,broken clouds
7,7,Thinadhoo,0.5333,72.9333,83.82,73,99,6.15,MV,overcast clouds
11,11,Arraial Do Cabo,-22.9661,-42.0278,75.87,87,0,21.70,BR,clear sky


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Magadi,IN,82.45,clear sky,12.9667,77.2333,
3,Butaritari,KI,81.45,broken clouds,3.0707,172.7902,
5,Panama City,PA,76.95,broken clouds,8.9936,-79.5197,
7,Thinadhoo,MV,83.82,overcast clouds,0.5333,72.9333,
11,Arraial Do Cabo,BR,75.87,clear sky,-22.9661,-42.0278,
21,Rikitea,PF,77.23,clear sky,-23.1203,-134.9692,
25,Vila Velha,BR,75.94,clear sky,-20.3297,-40.2925,
27,Yulara,AU,85.59,overcast clouds,-25.2406,130.9889,
28,Vaini,TO,78.30,broken clouds,-21.2000,-175.2000,
30,Simao,CN,83.34,clear sky,22.7886,100.9748,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Magadi,IN,82.45,clear sky,12.9667,77.2333,Gobi centre
3,Butaritari,KI,81.45,broken clouds,3.0707,172.7902,Isles Sunset Lodge
5,Panama City,PA,76.95,broken clouds,8.9936,-79.5197,Trump International Hotel & Tower Panama
11,Arraial Do Cabo,BR,75.87,clear sky,-22.9661,-42.0278,Pousada Porto Praia
21,Rikitea,PF,77.23,clear sky,-23.1203,-134.9692,People ThankYou
25,Vila Velha,BR,75.94,clear sky,-20.3297,-40.2925,Hotel Vitória Palace
27,Yulara,AU,85.59,overcast clouds,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
28,Vaini,TO,78.30,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
30,Simao,CN,83.34,clear sky,22.7886,100.9748,Changyu Hotel
33,Luanda,AO,76.75,overcast clouds,-8.8368,13.2343,EPIC SANA Luanda


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))